<a href="https://colab.research.google.com/github/ranianasser/ass.4/blob/main/MK_code_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.applications.mobilenet import MobileNet
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [7]:
batch_size = 32
img_size=224

train_generator=ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
        '/content/drive/MyDrive/MK/test',
target_size = (img_size, img_size),batch_size = batch_size)

test_generator=ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
    '/content/drive/MyDrive/MK/test',
target_size = (img_size, img_size),batch_size = batch_size)

Found 24 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
imgs,labels=next(train_generator)

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=0.10, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(21, 224, 224, 3)
(21, 2)
(3, 224, 224, 3)
(3, 2)


In [6]:
from keras.applications.vgg16 import VGG16
vgg16=VGG16()
model=Sequential()
for layer in vgg16.layers[:-1]:
 model.add(layer)

for layer in model.layers:
  layer.trainable = True # Not trainable weights
model.add(Dense(2, activation='softmax'))
model.summary()

553467096/553467096 [==============================] - 19s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                      

In [8]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [9]:
# Create an ImageDataGenerator object
data_augmentation = ImageDataGenerator(
    rotation_range=30,  #randomly rotated by up to 20 degrees in either direction.
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    zoom_range=0.3,  # set range for random zoom
    horizontal_flip=True,  # randomly flip images
    fill_mode='nearest'
)

train_data_augmentation = data_augmentation.flow(X_train, y_train)
validation_data_augmentation = data_augmentation.flow(X_val, y_val)

In [11]:
# Fit Model
early_stopping_monitor = EarlyStopping(patience = 4)
model.fit(train_data_augmentation, validation_data=validation_data_augmentation, epochs = 10, callbacks = [early_stopping_monitor])

Epoch 1/10
1/1 [==============================] - 53s 53s/step - loss: 1185.9552 - accuracy: 0.4762 - val_loss: 16.6611 - val_accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 53s 53s/step - loss: 25.5627 - accuracy: 0.4762 - val_loss: 1.2488 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 52s 52s/step - loss: 1.1944 - accuracy: 0.4762 - val_loss: 228.2652 - val_accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 51s 51s/step - loss: 160.0879 - accuracy: 0.5238 - val_loss: 10.8105 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 58s 58s/step - loss: 7.5723 - accuracy: 0.5238 - val_loss: 10.3005 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 52s 52s/step - loss: 7.4792 - accuracy: 0.5238 - val_loss: 2.3938 - val_accuracy: 0.3333


In [12]:
test_labels=test_generator.classes
predictions=np.argmax(model.predict(test_generator),axis=1)
print(predictions[:5])

1/1 [==============================] - 18s 18s/step
[0 0 0 0 0]


In [13]:
confusion_matrix(test_labels,predictions)

array([[12,  0],
       [12,  0]])

In [14]:
# Evaluate the model
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.5
Precision: 0.0
Recall: 0.0
F1-score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
